In [1]:
import pickle
import numpy as np
from nilearn import datasets
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import os
from graph import cal_graph

In [2]:
atlas_labels = datasets.fetch_atlas_aal()['labels']
# mpl.use("agg")

In [3]:
with open("FCs.pkl", "rb") as f:
    fcs:dict = pickle.load(f)

In [5]:
# 选择一个合适的二值化阈值
# 以全局效率为标准，绘制不同二值化阈值下全局效率统计学差异变化曲线
thresholds = range(35, 80, 2)
save_path = "graph_theory/cluster"
os.makedirs(save_path, exist_ok=True)
mw_stats = []
mw_p_values = []
for threshold in thresholds:
    index1 = []
    index2 = []
    for sub in fcs:
        if len(fcs[sub]["ses-preop"]) == 0:
            continue
        for run, fc in fcs[sub]["ses-preop"].items():
            # 创建图
            fcg = nx.Graph()
            fcg.add_nodes_from(atlas_labels)
            fcg.add_edges_from([(atlas_labels[i], atlas_labels[j]) for i in range(fc.shape[0]) for j in range(fc.shape[1]) if i != j and abs(fc[i][j]) >= threshold/100])
            if len(fcg.nodes) == 0:
                print(f"图为空：{sub}-ses-preop-{run}-{threshold}")
                # GE.append(GE[-1])
            else:
                index1.append(sum(nx.clustering(fcg).values()))
                # index1.append(nx.degree_assortativity_coefficient(fcg))
                if np.isnan(index1[-1]):
                    index1.pop()
        for run, fc in fcs[sub]["ses-postop"].items():
            # 创建图
            fcg = nx.Graph()
            fcg.add_nodes_from(atlas_labels)
            fcg.add_edges_from([(atlas_labels[i], atlas_labels[j]) for i in range(fc.shape[0]) for j in range(fc.shape[1]) if i != j and abs(fc[i][j]) >= threshold/100])
            if len(fcg.nodes) == 0:
                print(f"图为空：{sub}-ses-postop-{run}-{threshold}")
                # GE.append(GE[-1])
            else:
                index2.append(sum(nx.clustering(fcg).values()))
                # index2.append(nx.degree_assortativity_coefficient(fcg))
                if np.isnan(index2[-1]):
                    index2.pop()
    mw_stat, mw_p_value = stats.mannwhitneyu(index1, index2)
    mw_stats.append(mw_stat)
    mw_p_values.append(mw_p_value)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
fig.patch.set_color("white")
ax[0].set_title("mannwhitney u-stat")
ax[0].plot(thresholds, mw_stats)
ax[1].set_title("mannwhitney u p-value")
ax[1].plot(thresholds, mw_p_values)
fig.savefig(f"{save_path}/mw_stat.png", format="png")
plt.cla()
plt.clf()
plt.close("all")

In [52]:
threshold = 50

In [ ]:
# 计算图论参数并保存
save_path = "graph_theory/pkl"
os.makedirs(save_path, exist_ok=True)
graphs = {}
for sub in fcs:
    # if os.path.exists(f"{save_path}/{sub}.pkl"):
    #     continue
    fc_graph_theory= {}
    graphs[sub] = fc_graph_theory
    for ses in fcs[sub]:
        fc_graph_theory[ses] = {}
        for run, fc in fcs[sub][ses].items():

            # %%
            # 创建图
            fc_graph_theory[ses][run] = cal_graph(fc, threshold)
    with open(f"{save_path}/{sub}.pkl", "wb") as f:
        pickle.dump(fc_graph_theory, f)
with open(f"{save_path}/graphs.pkl", "wb") as f:
    pickle.dump(graphs, f)